#### Introduction

In this tutorial, we will write a Trace transformation to perform CPU Offloading of intermediate tensors. CPU Offloading is a technique to decrease the peak memory usage of the training loop. This can allow us to train a larger model which would otherwise won't be possible. However, we have to trade of some performance (increased training time) to achieve the same.

In [1]:
import gc

import torch
import torch.utils.benchmark

import thunder
from thunder import Transform
from thunder.core.proxies import TensorProxy, variableify
from thunder.core.pytree import tree_flatten, tree_map
from thunder.core.trace import tracectx, from_trace
from thunder.extend import OperatorExecutor
from thunder.core.symbol import BoundSymbol
from thunder.core import prims
from thunder.core.transforms import bsym_list_to_dag, Node, toposort_bsym_dag, TOPOSORT_ORDER
from thunder.core.vjp_utils import get_saved_for_backward_tensors

#### Transforms

`thunder` allows you to write your own transforms which can be applied at various stages during the compilation. To write our transform, we have to inherit from `thunder.Transform` class. This class implements the interface that each transform should have. By default, it provides no-op transformations.

We then provide our transform object to the `thunder.jit` via `transforms` argument.

However, before writing our transform, we will make an `OperatorExecutor` with which we will create 2 operators/symbol - 1. to offload tensors to CPU 2. Load the offloaded tensors back to CUDA device

You read more about adding custom operators in `adding_custom_operator.ipynb`.

In [2]:
# Create a new executor.
offload_exec = OperatorExecutor("offload_exec")


def offload_to_cpu_impl(t):
    # Due to https://github.com/Lightning-AI/lightning-thunder/issues/950
    # it may receive tensor on CPU.
    if t.device == torch.device("cpu"):
        return t

    packed = torch.empty(
        t.size(),
        dtype=t.dtype,
        layout=t.layout,
        pin_memory=True,
    )
    packed.copy_(t, non_blocking=False)
    return packed


offload_to_cpu = offload_exec.register_operator(
    "offload_to_cpu",
    meta=lambda t: TensorProxy("offloaded_" + t.name, like=t, device=thunder.core.devices.Device("cpu")),
    fn=offload_to_cpu_impl,
)


def load_to_gpu_impl(t, device):
    return t.to(device, non_blocking=True)


load_to_gpu = offload_exec.register_operator(
    "load_to_gpu",
    meta=lambda t, device: TensorProxy(like=t, device=thunder.core.devices.Device(device)),
    fn=load_to_gpu_impl,
)


Helper functions to implement our transformation

In [3]:
def _get_symbols_to_last_or_first_used_variables(symbols, first_used=False):
    variable_to_symbol = {}
    symbol_to_variables = {}

    def _mark_last_use(symbol, variable):
        if not variable in variable_to_symbol:
            variable_to_symbol[variable] = symbol
            symbol_to_variables.setdefault(symbol, []).append(variable)

    iter_symbols = symbols if first_used else reversed(symbols)
    for symbol in iter_symbols:
        # If this function is used in the combined nvfuser+torch executor, there are no symbols but regions.
        # Regions do not have args, kwargs
        if hasattr(symbol, "inputs"):
            variables = tuple(symbol.inputs) + tuple(symbol.outputs)
        else:
            variables = (symbol.flat_variableified_proxy_args) + tuple(symbol.flat_variableified_proxy_outs)
        tree_map(lambda x: _mark_last_use(symbol, x), variables)

    return symbol_to_variables, variable_to_symbol


def get_symbols_to_last_used_variables(symbols):
    return _get_symbols_to_last_or_first_used_variables(symbols)


def get_symbols_to_first_used_variables(symbols):
    return _get_symbols_to_last_or_first_used_variables(symbols, first_used=True)


def get_symbol_to_idx(symbols):
    '''
    This function returns a map from symbol to it's position in the list.
    '''
    return {sym: idx for idx, sym in enumerate(symbols)}


def move_closer_to_consumer(execution_trace):
    order_in_trace = {bsym: i for i, bsym in enumerate(execution_trace.bound_symbols)}

    def prefer_ops_closer_to_consumer(eligible_nodes: list[Node]) -> int:
        def key(node: Node) -> int:
            return order_in_trace[node.bsym]

        return min(range(len(eligible_nodes)), key=lambda i: key(eligible_nodes[i]))

    # This moves all del or clear collection at the bottom (as they don't return anything)
    bound_symbols = toposort_bsym_dag(
        bsym_list_to_dag(execution_trace.bound_symbols)[1],
        TOPOSORT_ORDER.BOTTOM_UP,
        selector=prefer_ops_closer_to_consumer,
    )

    for idx, bsym in enumerate(bound_symbols):
        if bsym.sym.id == prims.PrimIDs.DEL:
            break

    new_execution_trace = from_trace(execution_trace)
    new_execution_trace.bound_symbols = bound_symbols[:idx]

    new_execution_trace = thunder.executors.passes.del_last_used(new_execution_trace, clear_mutable_collections=True)
    return new_execution_trace

Now to the main topic, of writing the transform.

In [4]:
class CPUOffloading(Transform):
    '''
    Transform to implement CPU Offloading.

    save_tensor_policy - Users can pass a callback with signature fn(offloaded_tensors, forward_trace) to filter
                         the offloaded_tensors based on their preference eg. biggest 20% intermediate tensors or
                         intermediates of certain operations
    '''
    def __init__(self, save_tensor_policy=None):
        self.forward_pass = None
        self.backward_pass = None
        self._offloaded_tensors = ()
        self.save_tensor_policy = None
        if save_tensor_policy is not None:
            assert callable(save_tensor_policy)
            self.save_tensor_policy = save_tensor_policy

    def _get_tensors_to_offload(self, forward_trace):
        '''
        Based on the `forward_trace`, we find the symbols that we want to offload to CPU.
        This function finds the intermediate tensors that are saved for backward i.e. ones that are not input or output of this trace.
        '''
        return_bsym = forward_trace.bound_symbols[-1]
        trace_args = return_bsym.args[0]["flat_args"]
        saved_tensors = get_saved_for_backward_tensors(forward_trace)

        tensor_args_name = tuple(arg.name for arg in trace_args if isinstance(arg, TensorProxy))

        def is_in_tensor_args(t):
            return t.name in tensor_args_name

        def is_cuda_tensor(t):
            return t.device.type == "cuda"

        # Tensors which are intermediate and not argument to the computation trace are
        # the ones we are interested in offloading.
        tensors_to_offload = tuple(t for t in saved_tensors if ((not is_in_tensor_args(t)) and is_cuda_tensor(t)))

        # Apply users policy if present.
        if self.save_tensor_policy is not None:
            tensors_to_offload = self.save_tensor_policy(tensors_to_offload, forward_trace)
        self.tensors_to_offload = tensors_to_offload
        return self.tensors_to_offload

    def _replace_saved_tensors(self, forward_trace, new_output_map):
        return_bsym = forward_trace.bound_symbols[-1]
        return_bsym_args = return_bsym.args
        saved_tensors = return_bsym.args[1][0]

        new_saved_tensors = []
        for t in saved_tensors:
            new_output = new_output_map.get(variableify(t), t)
            new_saved_tensors.append(new_output)

        new_return_bsym = BoundSymbol.from_bsym(
            return_bsym, **{"args": (return_bsym_args[0], (tuple(new_saved_tensors), return_bsym_args[1][1]))}
        )

        # Replace the old return with our new return.
        forward_trace.bound_symbols.pop(-1)
        forward_trace.bound_symbols.append(new_return_bsym)

    def _offload_tensors_from_forward(self, computation_trace):
        '''
        This function takes the forward computation trace and performs following step
        1. Find the tensors to be offloaded using `_get_tensors_to_offload` (this also calls users `save_tensor_policy` if present).
        2. Insert calls to the `offload_to_cpu` symbol with the tensor to offload. These calls are placed after the last computational
           use of the tensors to be offloaded so that we free the memory as soon as possible.
        3. Finally, we update the last symbol i.e. `return` symbol to return the offloaded tensors instead of the original tensors.
        '''
        # Step 1
        # Find the tensors to offload.
        # We offload saved tensors which are not arguments to the computation trace and are saved for backwards.
        tensors_to_offload = self._get_tensors_to_offload(computation_trace)

        # Step 2
        # Insert the offloading calls after the last use of the saved tensor (which we want to offload).
        _, variable_to_last_symbol = get_symbols_to_last_used_variables(
            computation_trace.bound_symbols[:-1]
        )  # Ignore the return statement.
        symbol_to_idx = get_symbol_to_idx(computation_trace.bound_symbols)

        # Book keeping for backward pass update.
        new_output_map = {}
        new_output_dev_map = {}

        # Since we are inserting in the list (we need to obey increasing order) - else the insertions will be incorrect.
        sorted_tensors_to_offload = sorted(
            tensors_to_offload, key=lambda t: symbol_to_idx[variable_to_last_symbol[variableify(t)]]
        )
        for idx, t in enumerate(sorted_tensors_to_offload):
            last_used_symbol = variable_to_last_symbol[variableify(t)]
            last_used_symbol_idx = symbol_to_idx[last_used_symbol]
            computation_trace.push_scope([])
            with tracectx(computation_trace):
                o = offload_to_cpu(t)
                prims.python_del(t)
            scoped_comp = computation_trace.pop_scope()
            scoped_comp[0].header = "Created by CPU Offloading Transform"
            offload_to_cpu_symbol = scoped_comp[0]
            del_symbol = scoped_comp[1]

            # This will insert `del` first and then push it down when we insert `offload_to_cpu`.
            computation_trace.bound_symbols.insert(last_used_symbol_idx + 1 + (idx * 2), del_symbol)
            computation_trace.bound_symbols.insert(last_used_symbol_idx + 1 + (idx * 2), offload_to_cpu_symbol)

            # Update book keeping.
            new_output_map[variableify(t)] = o
            new_output_dev_map[variableify(t)] = t.device.device_str()

        # Step 3
        # Update the return symbol to return our offloaded tensors in saved for backward.
        self._replace_saved_tensors(computation_trace, new_output_map)

        # Book keeping for backward pass update.
        self._offloaded_tensors = new_output_map
        self._offloaded_tensors_dev = new_output_dev_map
        return computation_trace

    def _load_tensors_for_backward(self, computation_trace):
        '''
        This function takes the backward computation trace and performs following step
        1. Finds the unpack collection symbol which unpacks the saved tensors passed to the backward trace.
        2. Updates the unpack collection to unpack the offloaded tensors instead of the original ones.
        3. Before the first use of the offloaded tensor in computation, we insert the `load_to_gpu` to load the tensor back on GPU.
        '''
        self.backward_pass = computation_trace
        offloaded_tensors = self._offloaded_tensors
        offloaded_tensors_dev_map = self._offloaded_tensors_dev

        compute_producers, compute_consumers = thunder.core.utils.producers_and_consumers(computation_trace)

        # We want to insert `loads` before the first use of offloaded_tensors.
        _, variable_to_first_symbol = get_symbols_to_first_used_variables(computation_trace.bound_symbols)

        symbol_to_idx = get_symbol_to_idx(computation_trace.bound_symbols)

        # Step 1 and 2
        # Update unpack collection so that it
        # outputs the offloaded tensor proxies (not the original ones).
        unpack_sym = compute_producers[list(offloaded_tensors.keys())[0].proxy]
        unpack_idx = symbol_to_idx[unpack_sym]
        unpack_sym_out = unpack_sym.output
        new_out = []
        for out in unpack_sym_out:
            vout = variableify(out)
            if vout in offloaded_tensors:
                new_out.append(offloaded_tensors[vout])
            else:
                new_out.append(out)
        new_unpack_bsym = BoundSymbol.from_bsym(unpack_sym, output=tuple(new_out))
        computation_trace.bound_symbols[unpack_idx] = new_unpack_bsym

        # Now we again find the first usages of offloaded tensor
        # This will actually point us to the first consumer of the offloaded tensor.
        offset = unpack_idx + 1
        _, variable_to_first_symbol = get_symbols_to_first_used_variables(computation_trace.bound_symbols[offset:])

        # Step 3
        # Load the offloaded tensors to GPU before usage.
        # Should iterate in correct order (else insertion positions will be incorrect).
        for idx, (vt, offloaded_t) in enumerate(
            sorted(offloaded_tensors.items(), key=lambda kv: symbol_to_idx[variable_to_first_symbol[kv[0]]])
        ):
            first_used_symbol = variable_to_first_symbol[vt]
            first_used_symbol_idx = symbol_to_idx[first_used_symbol]
            t = vt.proxy
            device = offloaded_tensors_dev_map[vt]

            with tracectx(computation_trace):
                new_sym = load_to_gpu.bind(offloaded_t, device, output=t)

            new_sym.header = "Created by CPU Offloading Transform"
            computation_trace.bound_symbols.insert(first_used_symbol_idx + idx, new_sym)

        return computation_trace

    def transform_trace_post_optimization(self, computation_trace: thunder.TraceCtx, **kwargs):
        if self.forward_pass is None:
            self.forward_pass = computation_trace
            # Processing for the forward pass (only if we are going to compute backward).
            if "augmented_forward" in computation_trace.fn.__name__:
                computation_trace = self._offload_tensors_from_forward(computation_trace)
        else:
            # Skip if no tensor was offloaded.
            if len(self._offloaded_tensors) == 0:
                return computation_trace

            # We need this because in unmodified backward trace, the first consumer of saved_for_backward maybe
            # a reshape or permute op and the actual computation occurs 50-100 (or more) lines later.
            # Because of this we load more tensors than required eagerly (thus decreasing the memory gains from CPU Offloading).
            # This function is currently tailored to pattern observed in Llama-2
            # Eg. on line 92
            #   # Created by CPU Offloading Transform
            #   t1319 = load_to_gpu(offloaded_t1319, 'cuda:0')  # t1319: "cuda:0 f32[8, 1024, 11008]"
            #   t4021 = torch.reshape(t1319, (-1, 11008))  # t4021: "cuda:0 f32[8192, 11008]"
            #     # t4021 = ltorch.reshape(t1319, (-1, 11008))  # t4021: "cuda:0 f32[8192, 11008]"
            #       # t4021 = prims.reshape(t1319, (8192, 11008))  # t4021: "cuda:0 f32[8192, 11008]"
            #   del t1319
            # And it's usage in computation is at 612
            # t4022 = torch.matmul(t4020, t4021)  # t4022: "cuda:0 f32[4096, 11008]"
            #   t4022 = ltorch.matmul(t4020, t4021)  # t4022: "cuda:0 f32[4096, 11008]"
            #     t4022 = prims.matmul(t4020, t4021)  # t4022: "cuda:0 f32[4096, 11008]"
            computation_trace = move_closer_to_consumer(computation_trace)

            # Transform the backward trace to load offloaded tensors back to the device.
            computation_trace = self._load_tensors_for_backward(computation_trace)

        return computation_trace

In [5]:
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_accumulated_memory_stats()
    torch.cuda.reset_peak_memory_stats()
    print(f"Allocated Memory after cleaning {torch.cuda.memory_allocated() / 1e9} GB")

In [6]:
def benchmark(jmodel, model, args, kwargs):
    # NOTE - This function takes care of warm-up
    stmt = """
# Use the optimized model for prediction and backward
o = jmodel(*args, **kwargs)
o.sum().backward()
for param in model.parameters():  # use original model for clear grads
    param.grad = None
"""
    timer = torch.utils.benchmark.Timer(
        stmt=stmt, globals={"jmodel": jmodel, "model": model, "args": args, "kwargs": kwargs}
    ).timeit(number=10)
    return timer

Sanity Check the output values

In [18]:
class MySimpleModel(torch.nn.Module):
    def __init__(self, n_layers=10):
        super().__init__()
        self.fcs = torch.nn.ModuleList([torch.nn.Linear(16, 16) for _ in range(n_layers)])

    def forward(self, x):
        for fc in self.fcs:
            x = torch.nn.functional.relu(fc(x))
        
        return x

def get_model_and_args():
    device = 'cuda'
    model = MySimpleModel(n_layers=100).to(device)
    args = (torch.randn(128, 16, device=device),)
    kwargs = {}
    return model, args, kwargs

model, args, kwargs = get_model_and_args()

jmodel = thunder.jit(model, transforms=[CPUOffloading()])

actual = jmodel(*args, **kwargs)

# Verify that saved tensors are on CPU.
saved_tensor_devices = set()
for t in actual.grad_fn.saved_tensors:
    saved_tensor_devices.add(str(t.device))

assert "cpu" in saved_tensor_devices  # Verify that we actually have saved tensors on CPU

grad_output = torch.randn_like(actual)
actual_grads = torch.autograd.grad(actual, jmodel.parameters(), grad_output)



fw_traces = thunder.last_traces(jmodel)
bw_traces = thunder.last_backward_traces(jmodel)

with torch.no_grad():
    actual_cpu = actual.to("cpu")
    actual_grads_cpu = tree_map(lambda t: t.to("cpu"), actual_grads)

# Check against the eager model
expected = model(*args, **kwargs)
expected_grads = torch.autograd.grad(expected, model.parameters(), grad_output)

with torch.no_grad():
    expected_cpu = expected.to("cpu")
    expected_grads_cpu = tree_map(lambda t: t.to("cpu"), expected_grads)

torch.testing.assert_close(actual_cpu, expected_cpu)
torch.testing.assert_close(actual_grads_cpu, expected_grads_cpu)

del jmodel, model, args, kwargs, actual, actual_grads, expected, expected_grads, grad_output  # Free memory.

clear_memory()

Allocated Memory after cleaning 8.192e-06 GB


In [8]:
fw_traces[-1]

# Constructed by Delete Last Used (took 2 milliseconds)
import torch
import torch.nn.functional
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast
def augmented_forward_fn(x, t_fcs_0_bias, t_fcs_0_weight, t_fcs_1_bias, t_fcs_1_weight, t_fcs_2_bias, t_fcs_2_weight, t_fcs_3_bias, t_fcs_3_weight, t_fcs_4_bias, t_fcs_4_weight, t_fcs_5_bias, t_fcs_5_weight, t_fcs_6_bias, t_fcs_6_weight, t_fcs_7_bias, t_fcs_7_weight, t_fcs_8_bias, t_fcs_8_weight, t_fcs_9_bias, t_fcs_9_weight, t_fcs_10_bias, t_fcs_10_weight, t_fcs_11_bias, t_fcs_11_weight, t_fcs_12_bias, t_fcs_12_weight, t_fcs_13_bias, t_fcs_13_weight, t_fcs_14_bias, t_fcs_14_weight, t_fcs_15_bias, t_fcs_15_weight, t_fcs_16_bias, t_fcs_16_weight, t_fcs_17_bias, t_fcs_17_weight, t_fcs_18_bias, t_fcs_18_weight, t_fcs_19_bias, t_fcs_19_weight, t_fcs_20_bias, t_fcs_20_weight, t_fcs_21_bias, t_fcs_21_weight, t_fcs_22_bias, t_fcs_22_weight, t_fcs_23_bias, t_fcs_23_weight, t_fcs_24_bias, t_fcs_24_weight, t_fcs_25_bias, 

In [9]:
bw_traces[-1]

# Constructed by Delete Last Used (took 5 milliseconds)
import torch
from thunder.executors.torchex import no_autocast

@torch.no_grad()
@no_autocast
def backward_fn(saved_for_backward, cotangents):
  # saved_for_backward: "Collection"
  C0, _, = saved_for_backward
  clear_mutable_collection(saved_for_backward)
  del saved_for_backward
  offloaded_t1, offloaded_t10, offloaded_t100, offloaded_t102, offloaded_t105, \
  offloaded_t107, offloaded_t110, offloaded_t112, offloaded_t115, offloaded_t117, \
  offloaded_t12, offloaded_t120, offloaded_t122, offloaded_t125, offloaded_t127, \
  offloaded_t130, offloaded_t132, offloaded_t135, offloaded_t137, offloaded_t140, \
  offloaded_t142, offloaded_t145, offloaded_t147, offloaded_t15, offloaded_t150, \
  offloaded_t152, offloaded_t155, offloaded_t157, offloaded_t160, offloaded_t162, \
  offloaded_t165, offloaded_t167, offloaded_t17, offloaded_t170, offloaded_t172, \
  offloaded_t175, offloaded_t177, offloaded_t180, offloaded_t182, offloaded_t185

In [10]:
model, args, kwargs = get_model_and_args()

measurement_thunder = benchmark(thunder.jit(model), model, args, kwargs)
measurement_thunder_offload = benchmark(thunder.jit(model, transforms=[CPUOffloading()]), model, args, kwargs)

del model, args, kwargs

clear_memory()

Allocated Memory after cleaning 0.0 GB


In [11]:
measurement_thunder

stmt:
  # Use the optimized model for prediction and backward
  o = jmodel(*args, **kwargs)
  o.sum().backward()
  for param in model.parameters():  # use original model for clear grads
      param.grad = None

  8.79 ms
  1 measurement, 10 runs , 1 thread

In [12]:
measurement_thunder_offload

stmt:
  # Use the optimized model for prediction and backward
  o = jmodel(*args, **kwargs)
  o.sum().backward()
  for param in model.parameters():  # use original model for clear grads
      param.grad = None

  12.79 ms
  1 measurement, 10 runs , 1 thread

Let us try it on a real-life model Llama-3. We will run it on a smaller Llama-3.

In [13]:
from litgpt import Config, GPT
from functools import partial
from torch.testing import make_tensor

N_LAYER = 7
BLOCK_SIZE = 1024

def get_model_and_args(batchdims=8):
    with torch.device("cuda"):
        cfg: Config = Config.from_name("Llama-3-8B")
        # Smaller configuration
        cfg.n_layer = N_LAYER
        cfg.block_size = BLOCK_SIZE

        model = GPT(cfg)
        make = partial(make_tensor, low=0, high=255, device='cuda', dtype=torch.int64, requires_grad=False)
        shape = (batchdims,) + (cfg.block_size,)

        x = make(shape)
        args, kwargs = (x,), {}

    return model, args, kwargs, cfg

def print_memory_usage_and_benchmark(name):
    print(f"{name} took -")
    model, args, kwargs, cfg = get_model_and_args()

    if name == 'thunder':
        jmodel = thunder.jit(model)
    elif name == 'thunder_offload':
        jmodel = thunder.jit(model, transforms=[CPUOffloading()])
    else:
        raise RuntimeError("Received invalid value for `name` - try `thunder` or `thunder_offload`.")

    memory_after_model_load = torch.cuda.max_memory_allocated() / 1e9
    print(f"Peak memory after loading the model : {memory_after_model_load} GB")

    a = jmodel(*args, **kwargs)

    memory_after_forward = torch.cuda.max_memory_allocated() / 1e9
    print(f"Peak memory after forward the model : {memory_after_forward} GB")

    g = torch.rand_like(a)
    actual_grads = torch.autograd.grad(a, model.parameters(), g)

    memory_after_backward = torch.cuda.max_memory_allocated() / 1e9
    print(f"Peak memory after backward the model : {memory_after_backward} GB")

    del a, g, actual_grads  # Clear data which is not required for benchmark to free some memory.
    gc.collect()
    torch.cuda.empty_cache()

    measurement = benchmark(jmodel, model, args, kwargs)
    print(f"Benchmark Timings - mean : {measurement.mean} - median {measurement.median}")

    del jmodel, model, cfg, args, kwargs

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.reset_accumulated_memory_stats()
    print("CUDA Memory has been cleared and currently allocated memory is ", torch.cuda.memory_allocated() / 1e9)

/home/kkalambarkar/git/torch-vision/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [14]:
print_memory_usage_and_benchmark("thunder")

thunder took -
Peak memory after loading the model : 10.311422464 GB


[W814 12:14:19.087144933 sdp_utils.cpp:455] Warning: 1Torch was not compiled with flash attention. (function operator())
[W814 12:14:19.087159023 sdp_utils.cpp:504] Warning: 1Torch was not compiled with memory efficient attention. (function operator())


Peak memory after forward the model : 39.679967232 GB
Peak memory after backward the model : 47.024177152 GB
Benchmark Timings - mean : 3.944030920599471 - median 3.944030920599471
CUDA Memory has been cleared and currently allocated memory is  0.0


In [15]:
print_memory_usage_and_benchmark("thunder_offload")

thunder_offload took -
Peak memory after loading the model : 10.311422464 GB
Peak memory after forward the model : 14.64786944 GB
Peak memory after backward the model : 32.004145152 GB
Benchmark Timings - mean : 5.2520787032000955 - median 5.2520787032000955
CUDA Memory has been cleared and currently allocated memory is  0.0
